In [1]:
#RQ1: Step 1 - Setting up a prior belief based on targeted question

#targeted questions
question1 = "Do you prefer direct next steps or detailed discussion first?"
question1_options=["1. direct steps", "2. detailed discussion"]

question2= "Would you prefer a checklist or conversation?"
question2_options=["1. checklist", "2. conversation"]

In [2]:
#Function for asking question

def ask_question(question, answer):
    #question 1
    print(question)
    for i in range(len(answer)):
        print(answer[i])
    choice= input('Enter your preferred answer number') #change it to better UI based button later
    return answer[int(choice)-1]


choice_result = []

choice_result.append(ask_question(question1, question1_options))
choice_result.append(ask_question(question2, question2_options))

print(choice_result)


Do you prefer direct next steps or detailed discussion first?
1. direct steps
2. detailed discussion
Would you prefer a checklist or conversation?
1. checklist
2. conversation
['1. direct steps', '1. checklist']


In [3]:
#creating user profile based on user's answer

user_profile = {"preferred_conversation_style": None,
                "ph1": None,
                "ph2": None}

#take 2 answers and based on that assign key-value pair for user profile--think


if choice_result[0]=="1. direct steps" and choice_result[1] == "1. checklist":
    user_profile["preferred_conversation_style"] = "action_based"
elif choice_result[0]=="2. detailed discussion" and choice_result[1] == "2. conversation":
    user_profile["preferred_conversation_style"] = "relationship_based"
else:
    user_profile["preferred_conversation_style"] = "mixed"
        
print(user_profile)


{'preferred_conversation_style': 'action_based', 'ph1': None, 'ph2': None}


In [4]:
#assign prior belief value - hardcoded, might change with an LLM prompt layer (step 1 completed)

if user_profile["preferred_conversation_style"] == "action_based":
    user_profile["ph1"] = .90
    user_profile["ph2"] = .10
elif user_profile["preferred_conversation_style"] == "relationship_based":
    user_profile["ph1"] = .10
    user_profile["ph2"] = .90
else:
    user_profile["ph1"]=.60
    user_profile["ph2"]=.40
    
print(user_profile)

{'preferred_conversation_style': 'action_based', 'ph1': 0.9, 'ph2': 0.1}


In [5]:
#RQ1: Step 2 - Observing behavior with LLM prompt and treating each message like evidance P(E)

user_input = input("Hi there! What you want to chat about?")

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

d:\Coding\GitHub Repos\Personalized-LLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
torch.set_default_device("cuda")


In [8]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.69s/it]


In [ ]:
inputs = tokenizer(f'''You are a conversation analyst. Estimate how likely it is that the user is Action Based or Relationship Based. Give me peh1 and peh2 values where peh1 is probability user would say that if they are action based and peh2 is probability user would say that if they are relationship based. Here is the user input: {user_input}''', return_tensors="pt", return_attention_mask=False)

# add prompt engioneering to close the loop 


# MOVE inputs to model's device
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()} 

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a conversation analyst. Estimate how likely it is that the user is Action Based or Relationship Based. Give me peh1 and peh2 values where peh1 is probability user would say that if they are action based and peh2 is probability user would say that if they are relationship based. Here is the user input: About problem solving, I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I think it is important to be able to think outside the box. I
